### Fetching Facebook page data, posts and post engagement

**Importing `facebook-sdk` (python package for Facebook) and `python-decouple` for environment variables**

In [1]:
import facebook
from decouple import config

**Setting up `pymongo`, a package used to connect with `MongoDB`, where the posts will be stored**

In [2]:
import pymongo
db_client = pymongo.MongoClient()
pages_db = db_client["fbpages"]
doc = pages_db["posts"]

**Importing the environment variables from an `env` file**

In [3]:
page_token = config('PAGE_ACCESS_TOKEN')
page_id = config('PAGE_ID')

**Fetching page info from Facebook**

In [4]:
graph = facebook.GraphAPI(access_token=page_token, version="3.1")
page_info = graph.get_object(id=page_id, fields='about, website')
page_info

{'about': 'Writing my way through life', 'id': '105141914536236'}

**Fetching posts and connected data for the given page**

In [5]:
post_data = graph.get_object(id=page_id, fields='posts.limit(100)')
post_data

{'posts': {'data': [{'created_time': '2020-05-13T08:17:35+0000',
    'message': 'Lockdown can either be boring and sad, or it can be your chance to rest, have fun, and do all the stuff you previously had no time for. Any ideas for lockdown? Share some in the comments.',
    'id': '105141914536236_107376184312809'},
   {'created_time': '2020-05-13T08:14:16+0000',
    'message': 'These COVID-19 times will teach us the true power of saving. Yes, we have all heard that investing is better than saving, but this crisis has showed us that it is important to have some saved cash to fall back to. #Covid19 #Survival #Saving',
    'id': '105141914536236_107373617646399'},
   {'created_time': '2020-05-11T06:37:46+0000',
    'message': 'Welcome to Sanyu Means Joy!',
    'id': '105141914536236_105150181202076'}],
  'paging': {'cursors': {'before': 'QVFIUjU4Sk5ETkt5QTQ5MDZArR1VYTy1EenFJTVlMTDdfWDFrSWMtcGRvOTVYaWdIRGNSYk9lYTFXMjlpQUhUbnN4V0Ntc1dZAWXZAOSXhZAalVCV1BKWEVBejNhUkpSS25WWFRtdXZAaZAHFtRzRJYjV

In [6]:
posts = post_data['posts']['data']
for post in posts:
    print(f'{post}\n')
    post_comments = graph.get_connections(id=post['id'],           
                                        connection_name='comments',
                                        include_hidden=True,       
                                        filter='stream',           
                                        summary='total_count')
    print(f'{post_comments}\n')


{'created_time': '2020-05-13T08:17:35+0000', 'message': 'Lockdown can either be boring and sad, or it can be your chance to rest, have fun, and do all the stuff you previously had no time for. Any ideas for lockdown? Share some in the comments.', 'id': '105141914536236_107376184312809'}

{'data': [{'created_time': '2020-05-13T09:30:26+0000', 'message': 'No ideas, however long the list is can occupy you for all the lockdown', 'id': '107376184312809_107416554308772'}, {'created_time': '2020-05-13T09:30:52+0000', 'message': 'You have to let life take its course 😂', 'id': '107376184312809_107416700975424'}, {'created_time': '2020-05-13T09:35:06+0000', 'from': {'name': 'Sanyu Means Joy', 'id': '105141914536236'}, 'message': "Hahaha......you're right", 'id': '107376184312809_107419607641800'}], 'paging': {'cursors': {'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEEzTkRFMk5UVTBNekE0TnpjeU9qRTFPRGt6TmpJeU1qYz0ZD', 'after': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZANVEEzTkRFNU5qQTNOalF4T0RBd09qRTFPRGt6TmpJMU1E

**Inserting the posts into a `MongoDB` database**

In [ ]:
doc.insert_many(posts)